To install qiskit and environment setup, visit: https://learn.qiskit.org/course/ch-prerequisites/environment-setup-guide-to-work-with-qiskit-textbook

In [1]:
from qiskit import QuantumCircuit

In [2]:
# create qc w/ 3 qubits (or 3 quantum registers) and 3 classical bits (or 3 classical registers)
qc = QuantumCircuit(3, 3)
qc.draw()

q_0: 
     
q_1: 
     
q_2: 
     
c: 3/

https://qiskit.org/documentation/stubs/qiskit.circuit.QuantumCircuit.html

In [3]:
# add measuring gates
qc.measure(qubit = [0,1,2], cbit = [0,1,2])
qc.draw()

┌─┐      
q_0: ┤M├──────
     └╥┘┌─┐   
q_1: ─╫─┤M├───
      ║ └╥┘┌─┐
q_2: ─╫──╫─┤M├
      ║  ║ └╥┘
c: 3/═╩══╩══╩═
      0  1  2

In [4]:
# run the circuit
from qiskit.providers.aer import AerSimulator
sim = AerSimulator() 

https://qiskit.org/documentation/stubs/qiskit.providers.aer.AerSimulator.html

In [5]:
job = sim.run(qc) # run the experiment
result = job.result() # get the result
result.get_counts() # interpret the results as a "counts" dict

{'000': 1024}

Result:

keys -> bit-strings representing the combination of qubits being measured<br>
values -> the number of times that bit-string was measured

we can see that all qubits start as state '0'

we're gonna need to get result from quantum circuit very often, so i'll write a function for it

In [6]:
def getQCResult(qc):
    sim = AerSimulator() 
    job = sim.run(qc) 
    result = job.result()
    print("Result: ", result.get_counts())

## Encoding an input

In [7]:
qc = QuantumCircuit(3, 3)
qc.x([0,1])  # Perform X-gates on qubits 0 & 1
qc.measure([0,1,2], [0,1,2])
qc.draw()    # returns a drawing of the circuit

┌───┐┌─┐   
q_0: ┤ X ├┤M├───
     ├───┤└╥┘┌─┐
q_1: ┤ X ├─╫─┤M├
     └┬─┬┘ ║ └╥┘
q_2: ─┤M├──╫──╫─
      └╥┘  ║  ║ 
c: 3/══╩═══╩══╩═
       2   0  1

In [8]:
getQCResult(qc)

Result:  {'011': 1024}


'011' -> 3

### Exercise 1: encode 5

In [9]:
qc = QuantumCircuit(3, 3)
qc.x([0,2]) 
qc.measure([0,1,2], [0,1,2])
qc.draw()

┌───┐   ┌─┐   
q_0: ┤ X ├───┤M├───
     └───┘┌─┐└╥┘   
q_1: ─────┤M├─╫────
     ┌───┐└╥┘ ║ ┌─┐
q_2: ┤ X ├─╫──╫─┤M├
     └───┘ ║  ║ └╥┘
c: 3/══════╩══╩══╩═
           1  0  2

In [10]:
getQCResult(qc)

Result:  {'101': 1024}


let's try to mix the order of NOT gates (parameter of x method) and Measure gates (parameter of measure method)

In [11]:
qc = QuantumCircuit(3, 3)
qc.x([2,0]) 
qc.measure([2,0,1], [2,0,1])
qc.draw()

┌───┐      ┌─┐
q_0: ┤ X ├──────┤M├
     └───┘┌─┐   └╥┘
q_1: ─────┤M├────╫─
     ┌───┐└╥┘┌─┐ ║ 
q_2: ┤ X ├─╫─┤M├─╫─
     └───┘ ║ └╥┘ ║ 
c: 3/══════╩══╩══╩═
           1  2  0

In [12]:
getQCResult(qc)

Result:  {'101': 1024}


the simulation result is consistent but somehow the order of Measure gates in the result of draw method is not the same - which is understandable in itself but can't think of the rule of the order based on just the method parameters.

### Exercise2: encode 4, by modifying above circuit

In [13]:
qc.x([0])
qc.measure([0,1,2],[0,1,2])
qc.draw()

┌───┐      ┌─┐┌───┐   ┌─┐
q_0: ┤ X ├──────┤M├┤ X ├───┤M├
     └───┘┌─┐   └╥┘└┬─┬┘   └╥┘
q_1: ─────┤M├────╫──┤M├─────╫─
     ┌───┐└╥┘┌─┐ ║  └╥┘ ┌─┐ ║ 
q_2: ┤ X ├─╫─┤M├─╫───╫──┤M├─╫─
     └───┘ ║ └╥┘ ║   ║  └╥┘ ║ 
c: 3/══════╩══╩══╩═══╩═══╩══╩═
           1  2  0   1   2  0

In [14]:
getQCResult(qc)

Result:  {'100': 1024}


### Experiment: messing with Measure gates order

In [15]:
# messing with measure
qc = QuantumCircuit(3, 3)
qc.x([1])
qc.measure([0,1,2],[0,2,1])
qc.draw()

┌─┐   
q_0: ─────┤M├───
     ┌───┐└╥┘┌─┐
q_1: ┤ X ├─╫─┤M├
     └┬─┬┘ ║ └╥┘
q_2: ─┤M├──╫──╫─
      └╥┘  ║  ║ 
c: 3/══╩═══╩══╩═
       1   0  2

In [16]:
getQCResult(qc)

Result:  {'100': 1024}


As expected - the 1st and 2nd qubit is measured in switched order

## XOR: controlled-not-gate

In [17]:
# Create quantum circuit with 2 qubits and 2 classical bits
qc = QuantumCircuit(2, 2)
qc.x(0)
qc.cx(control_qubit = 0, target_qubit = 1)  # CNOT controlled by qubit 0 and targeting qubit 1
qc.measure([0,1], [0,1])
display(qc.draw())     # display a drawing of the circuit

┌───┐     ┌─┐   
q_0: ┤ X ├──■──┤M├───
     └───┘┌─┴─┐└╥┘┌─┐
q_1: ─────┤ X ├─╫─┤M├
          └───┘ ║ └╥┘
c: 2/═══════════╩══╩═
                0  1

In [18]:
getQCResult(qc)

Result:  {'11': 1024}


The control qubit is unaffected, while the target qubit is 'overwritten' by CNOT/XOR gate

What if we don't want to overwrite a qubit and store the result somewhere else?

Add a new qubit

In [19]:
qc = QuantumCircuit(3, 1)
qc.x(1) 
qc.x(0)
qc.cx(0,2)
qc.cx(1,2)
qc.measure([2],[0])
display(qc.draw())

┌───┐             
q_0: ┤ X ├──■──────────
     ├───┤  │          
q_1: ┤ X ├──┼────■─────
     └───┘┌─┴─┐┌─┴─┐┌─┐
q_2: ─────┤ X ├┤ X ├┤M├
          └───┘└───┘└╥┘
c: 1/════════════════╩═
                     0

In [20]:
getQCResult(qc)

Result:  {'0': 1024}


In [21]:
 def XOR(x, y):
    qc = QuantumCircuit(3, 1)
    if x:
        qc.x(0)
    if y:
        qc.x(1)
    qc.cx(0,2)
    qc.cx(1,2)
    qc.measure([2],[0])
    getQCResult(qc)

In [22]:
XOR(0,0)

Result:  {'0': 1024}


In [23]:
XOR(0,1)

Result:  {'1': 1024}


In [24]:
XOR(1,0)

Result:  {'1': 1024}


In [25]:
XOR(1,1)

Result:  {'0': 1024}


## Adder - using Toffoli (AND) gate

In [26]:
qc = QuantumCircuit(4, 2)
# input
qc.x(0)
qc.x(1)
# last digit
qc.cx(0,2)
qc.cx(1,2)
# front digit
qc.ccx(0,1,3) # Toffoli gate
qc.measure([2,3],[0,1])
display(qc.draw())

┌───┐                     
q_0: ┤ X ├──■─────────■────────
     ├───┤  │         │        
q_1: ┤ X ├──┼────■────■────────
     └───┘┌─┴─┐┌─┴─┐  │  ┌─┐   
q_2: ─────┤ X ├┤ X ├──┼──┤M├───
          └───┘└───┘┌─┴─┐└╥┘┌─┐
q_3: ───────────────┤ X ├─╫─┤M├
                    └───┘ ║ └╥┘
c: 2/═════════════════════╩══╩═
                          0  1

In [27]:
getQCResult(qc)

Result:  {'10': 1024}


In [28]:
def Add(x, y):
    qc = QuantumCircuit(4, 2)
    # input
    if x:
        qc.x(0)
    if y:
        qc.x(1)
    # last digit
    qc.cx(0,2)
    qc.cx(1,2)
    # front digit
    qc.ccx(0,1,3)
    qc.measure([2,3],[0,1])
    getQCResult(qc)

In [29]:
Add(0,0)

Result:  {'00': 1024}


In [30]:
Add(0,1)

Result:  {'01': 1024}


In [31]:
Add(1,0)

Result:  {'01': 1024}


In [32]:
Add(1,1)

Result:  {'10': 1024}
